# 🧭 TSP com QAOA: Implementação Didática
## Alinhada com a Formulação Matemática

Este notebook implementa o Problema do Caixeiro Viajante (TSP) usando QAOA, seguindo **exatamente** a formulação matemática proposta:

1. **Formulação Clássica** com variáveis $x_{i,t}$
2. **Conversão QUBO** com penalidades $H_{p1}$ e $H_{p2}$
3. **Mapeamento Quântico** $x_{i,t} \rightarrow \frac{I - \hat{Z}_{i,t}}{2}$
4. **QAOA** com operadores $U_C(\gamma)$ e $U_M(\beta)$

---
## 📦 Célula 1: Instalação das Dependências

In [ ]:
# Descomente para instalar (execute apenas uma vez)

# !pip install qiskit==1.2.4
# !pip install qiskit-optimization==0.6.1
# !pip install qiskit-algorithms==0.3.0
# !pip install qiskit-aer==0.15.1
# !pip install numpy matplotlib

---
## 📚 Célula 2: Importações

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations
import time
import warnings
warnings.filterwarnings('ignore')

# Qiskit imports
try:
    from qiskit_optimization import QuadraticProgram
    from qiskit_optimization.converters import QuadraticProgramToQubo
    from qiskit_optimization.translators import from_docplex_mp
    from qiskit_algorithms import QAOA
    from qiskit_algorithms.optimizers import COBYLA, SPSA
    from qiskit.primitives import Sampler
    from qiskit.quantum_info import SparsePauliOp, Statevector
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    QISKIT_AVAILABLE = True
    print("✅ Qiskit instalado e funcionando!")
except ImportError as e:
    QISKIT_AVAILABLE = False
    print(f"❌ Erro ao importar Qiskit: {e}")
    print("Descomente a célula de instalação e reinicie o kernel.")

---
## 📊 Célula 3: Matrizes de Distância

Definimos a matriz de distâncias $d_{ij}$ que representa o custo de ir da cidade $i$ para a cidade $j$.

In [ ]:
# Matrizes de distâncias para 3 a 6 cidades
graphs = {
    3: np.array([
        [0, 10, 15],
        [10, 0, 20],
        [15, 20, 0]
    ], dtype=float),
    
    4: np.array([
        [0, 1, 50, 50],
        [1, 0, 2, 50],
        [50, 2, 0, 3],
        [50, 50, 3, 0]
    ], dtype=float),
    
    5: np.array([
        [0, 2, 9, 10, 7],
        [1, 0, 6, 4, 3],
        [15, 7, 0, 8, 3],
        [6, 3, 12, 0, 11],
        [9, 7, 5, 6, 0]
    ], dtype=float),
    
    6: np.array([
        [0, 3, 6, 7, 8, 9],
        [3, 0, 5, 6, 7, 8],
        [6, 5, 0, 4, 5, 6],
        [7, 6, 4, 0, 3, 4],
        [8, 7, 5, 3, 0, 2],
        [9, 8, 6, 4, 2, 0]
    ], dtype=float)
}

# Exibe as matrizes
for n, D in graphs.items():
    print(f"\n📍 Matriz D (n={n} cidades):")
    print(D.astype(int))

---
## 🔢 Célula 4: Formulação Matemática Clássica

### Variável de Decisão $x_{i,t}$

$$x_{i,t} = \begin{cases} 1 & \text{se a cidade } i \text{ é visitada no passo } t \\ 0 & \text{caso contrário} \end{cases}$$

### Função Objetivo (Distância Total)

$$C(\mathbf{x}) = \sum_{i,j=0}^{n-1} \sum_{t=0}^{n-1} d_{ij} \, x_{i,t} \, x_{j, (t+1) \mod n}$$

### Restrições

1. Cada cidade $i$ é visitada em exatamente um instante $t$:
$$\sum_{t=0}^{n-1} x_{i,t} = 1, \quad \forall i$$

2. Em cada instante $t$, exatamente uma cidade é visitada:
$$\sum_{i=0}^{n-1} x_{i,t} = 1, \quad \forall t$$

In [ ]:
def indice_qubit(i, t, n):
    """
    Mapeia (cidade i, tempo t) para índice linear do qubit.
    
    Para n cidades, temos n² qubits organizados como:
    qubit[i*n + t] = x_{i,t}
    """
    return i * n + t


def mostrar_mapeamento_variaveis(n):
    """
    Exibe o mapeamento das variáveis x_{i,t} para qubits.
    """
    print(f"\n🔢 Mapeamento de Variáveis x_{{i,t}} para {n} cidades ({n}² = {n**2} qubits):")
    print("="*60)
    
    header = "        " + "".join([f"t={t:<6}" for t in range(n)])
    print(header)
    print("-"*60)
    
    for i in range(n):
        linha = f"cidade {i}: "
        for t in range(n):
            idx = indice_qubit(i, t, n)
            linha += f"q[{idx}]   "
        print(linha)
    
    print("\nOnde q[k] representa o k-ésimo qubit do circuito.")


# Demonstração para n=3
mostrar_mapeamento_variaveis(3)

---
## 📐 Célula 5: Construção QUBO Explícita

### Termos de Penalidade

As restrições são incorporadas via penalidades quadráticas:

$$H_{p1} = A \sum_{i=0}^{n-1} \left( \sum_{t=0}^{n-1} x_{i,t} - 1 \right)^2$$

$$H_{p2} = A \sum_{t=0}^{n-1} \left( \sum_{i=0}^{n-1} x_{i,t} - 1 \right)^2$$

### Hamiltoniano QUBO Completo

$$H_{\text{QUBO}} = \underbrace{\sum_{i,j,t} d_{ij} \, x_{i,t} \, x_{j,(t+1)\mod n}}_{H_{\text{dist}}} + H_{p1} + H_{p2}$$

O fator de penalidade $A$ deve ser **maior que a maior distância possível** para garantir que soluções inválidas tenham custo proibitivo.

In [ ]:
def construir_qubo_manual(D, penalty=None):
    """
    Constrói a matriz QUBO manualmente seguindo a formulação teórica.
    
    Parâmetros:
    -----------
    D : np.array
        Matriz de distâncias (n x n)
    penalty : float
        Fator de penalidade A. Se None, usa max(D) * n + 1
    
    Retorna:
    --------
    Q : np.array
        Matriz QUBO (n² x n²)
    A : float
        Fator de penalidade usado
    """
    n = len(D)
    num_qubits = n * n
    
    # Fator de penalidade A (deve ser > soma máxima possível de distâncias)
    if penalty is None:
        A = np.max(D) * n + 1
    else:
        A = penalty
    
    # Inicializa matriz QUBO
    Q = np.zeros((num_qubits, num_qubits))
    
    print(f"\n🔧 Construindo QUBO para n={n} cidades")
    print(f"   Número de qubits: {num_qubits}")
    print(f"   Fator de penalidade A = {A}")
    print(f"   (max(D) = {np.max(D)}, garantindo A > qualquer rota válida)")
    
    # =========================================
    # TERMO 1: H_dist (custo das distâncias)
    # =========================================
    # H_dist = Σ_{i,j,t} d_{ij} * x_{i,t} * x_{j,(t+1) mod n}
    
    print("\n   📍 Construindo H_dist (termo de distância)...")
    
    for i in range(n):
        for j in range(n):
            if i != j:  # Ignora diagonal (d_ii = 0)
                for t in range(n):
                    t_next = (t + 1) % n
                    idx1 = indice_qubit(i, t, n)
                    idx2 = indice_qubit(j, t_next, n)
                    
                    # Termo quadrático: d_{ij} * x_{i,t} * x_{j,t+1}
                    Q[idx1, idx2] += D[i, j]
    
    # =========================================
    # TERMO 2: H_p1 (cada cidade visitada uma vez)
    # =========================================
    # H_p1 = A * Σ_i (Σ_t x_{i,t} - 1)²
    #      = A * Σ_i [Σ_t x_{i,t}² + 2*Σ_{t<t'} x_{i,t}*x_{i,t'} - 2*Σ_t x_{i,t} + 1]
    
    print("   📍 Construindo H_p1 (restrição: cada cidade visitada uma vez)...")
    
    for i in range(n):
        for t in range(n):
            idx = indice_qubit(i, t, n)
            # Termo linear: -2A (de expandir (Σx - 1)²)
            # Como x² = x para variáveis binárias: x_{i,t}² - 2*x_{i,t} = -x_{i,t}
            Q[idx, idx] += A * (1 - 2)  # = -A
            
            # Termos quadráticos: 2A * x_{i,t} * x_{i,t'} para t ≠ t'
            for t2 in range(t + 1, n):
                idx2 = indice_qubit(i, t2, n)
                Q[idx, idx2] += 2 * A
    
    # =========================================
    # TERMO 3: H_p2 (cada tempo tem uma cidade)
    # =========================================
    # H_p2 = A * Σ_t (Σ_i x_{i,t} - 1)²
    
    print("   📍 Construindo H_p2 (restrição: cada tempo tem uma cidade)...")
    
    for t in range(n):
        for i in range(n):
            idx = indice_qubit(i, t, n)
            # Termo linear
            Q[idx, idx] += A * (1 - 2)  # = -A
            
            # Termos quadráticos: 2A * x_{i,t} * x_{i',t} para i ≠ i'
            for i2 in range(i + 1, n):
                idx2 = indice_qubit(i2, t, n)
                Q[idx, idx2] += 2 * A
    
    # Constante (não afeta otimização, mas completa a energia)
    constante = 2 * n * A  # De expandir os (Σx - 1)²
    
    print(f"\n   ✅ QUBO construída! Shape: {Q.shape}")
    print(f"   ✅ Constante aditiva: {constante}")
    
    return Q, A, constante


# Demonstração para n=3
Q_demo, A_demo, const_demo = construir_qubo_manual(graphs[3])

In [ ]:
def visualizar_matriz_qubo(Q, n):
    """
    Visualiza a matriz QUBO com anotações.
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    im = ax.imshow(Q, cmap='RdBu_r', aspect='equal')
    plt.colorbar(im, ax=ax, label='Coeficiente QUBO')
    
    # Labels
    labels = [f"x_{{{i},{t}}}" for i in range(n) for t in range(n)]
    ax.set_xticks(range(len(labels)))
    ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right', fontsize=8)
    ax.set_yticklabels(labels, fontsize=8)
    
    ax.set_title(f'Matriz QUBO para TSP com {n} cidades\n'
                 f'$H_{{QUBO}} = H_{{dist}} + H_{{p1}} + H_{{p2}}$', fontsize=12)
    ax.set_xlabel('Variável $x_{j,t}$')
    ax.set_ylabel('Variável $x_{i,t}$')
    
    # Grade para separar blocos de cidades
    for k in range(1, n):
        ax.axhline(y=k*n - 0.5, color='black', linewidth=0.5)
        ax.axvline(x=k*n - 0.5, color='black', linewidth=0.5)
    
    plt.tight_layout()
    return fig


# Visualiza QUBO para n=3
fig = visualizar_matriz_qubo(Q_demo, 3)
plt.show()

---
## ⚛️ Célula 6: Mapeamento para Hamiltoniano Quântico

### Transformação de Variáveis Binárias para Operadores de Pauli

O mapeamento das variáveis clássicas $x_{i,t} \in \{0,1\}$ para operadores quânticos é realizado por:

$$x_{i,t} \longrightarrow \hat{n}_{i,t} = \frac{I - \hat{Z}_{i,t}}{2}$$

Onde:
- $I$ é o operador identidade
- $\hat{Z}_{i,t}$ é o operador Pauli-Z atuando no qubit $(i,t)$

Esta transformação mapeia:
- Estado $|0\rangle$ (autovalor $Z=+1$) $\rightarrow$ $x=0$
- Estado $|1\rangle$ (autovalor $Z=-1$) $\rightarrow$ $x=1$

### Hamiltoniano de Custo Quântico

$$\hat{H}_C = \hat{H}_{\text{dist}} + \hat{H}_{p1} + \hat{H}_{p2}$$

In [ ]:
def qubo_para_ising(Q, constante=0):
    """
    Converte matriz QUBO para Hamiltoniano Ising (Pauli-Z).
    
    Usa a transformação: x = (1 - Z) / 2
    
    QUBO: H = Σ_i Q_ii * x_i + Σ_{i<j} Q_ij * x_i * x_j
    Ising: H = Σ_i h_i * Z_i + Σ_{i<j} J_ij * Z_i * Z_j + offset
    
    Retorna:
    --------
    h : np.array
        Coeficientes lineares (campos locais)
    J : np.array
        Coeficientes quadráticos (acoplamentos)
    offset : float
        Constante de energia
    """
    n = len(Q)
    
    # Inicializa
    h = np.zeros(n)
    J = np.zeros((n, n))
    offset = constante
    
    # Processa termos diagonais (lineares em x)
    for i in range(n):
        # x_i = (1 - Z_i) / 2
        # Q_ii * x_i = Q_ii * (1 - Z_i) / 2 = Q_ii/2 - Q_ii/2 * Z_i
        offset += Q[i, i] / 2
        h[i] -= Q[i, i] / 2
    
    # Processa termos fora da diagonal (quadráticos em x)
    for i in range(n):
        for j in range(i + 1, n):
            q_ij = Q[i, j] + Q[j, i]  # Simetriza
            if q_ij != 0:
                # x_i * x_j = (1-Z_i)/2 * (1-Z_j)/2 
                #           = (1 - Z_i - Z_j + Z_i*Z_j) / 4
                offset += q_ij / 4
                h[i] -= q_ij / 4
                h[j] -= q_ij / 4
                J[i, j] += q_ij / 4
    
    return h, J, offset


def criar_hamiltoniano_pauli(h, J, n_cidades):
    """
    Cria o Hamiltoniano como soma de operadores Pauli.
    
    H_C = Σ_i h_i * Z_i + Σ_{i<j} J_ij * Z_i * Z_j
    """
    n_qubits = len(h)
    pauli_list = []
    
    # Termos de campo local (Z_i)
    for i in range(n_qubits):
        if abs(h[i]) > 1e-10:
            # Cria string Pauli: 'I...IZI...I' com Z na posição i
            pauli_str = ['I'] * n_qubits
            pauli_str[n_qubits - 1 - i] = 'Z'  # Qiskit usa ordem reversa
            pauli_list.append((''.join(pauli_str), h[i]))
    
    # Termos de acoplamento (Z_i * Z_j)
    for i in range(n_qubits):
        for j in range(i + 1, n_qubits):
            if abs(J[i, j]) > 1e-10:
                pauli_str = ['I'] * n_qubits
                pauli_str[n_qubits - 1 - i] = 'Z'
                pauli_str[n_qubits - 1 - j] = 'Z'
                pauli_list.append((''.join(pauli_str), J[i, j]))
    
    return pauli_list


# Demonstração
print("\n⚛️ Conversão QUBO → Ising para n=3 cidades:")
print("="*60)

h_demo, J_demo, offset_demo = qubo_para_ising(Q_demo, const_demo)

print(f"\nCampos locais h (coeficientes de Z_i):")
for i in range(len(h_demo)):
    if abs(h_demo[i]) > 1e-10:
        cidade = i // 3
        tempo = i % 3
        print(f"   h[{i}] = {h_demo[i]:.2f}  (qubit x_{{{cidade},{tempo}}})")

print(f"\nOffset (constante): {offset_demo:.2f}")
print(f"\nNúmero de termos de acoplamento J_ij não-nulos: {np.count_nonzero(J_demo)}")

In [ ]:
def mostrar_hamiltoniano_resumido(h, J, n_cidades, max_termos=10):
    """
    Mostra o Hamiltoniano em formato legível.
    """
    n_qubits = n_cidades ** 2
    
    print(f"\n📜 Hamiltoniano de Custo $\\hat{{H}}_C$ (primeiros {max_termos} termos):")
    print("="*70)
    print("\n$\\hat{H}_C = $")
    
    termos = []
    
    # Termos Z_i
    for i in range(n_qubits):
        if abs(h[i]) > 1e-10:
            cidade = i // n_cidades
            tempo = i % n_cidades
            sinal = "+" if h[i] > 0 else "-"
            termos.append(f"{sinal} {abs(h[i]):.2f} Z_{{{cidade},{tempo}}}")
    
    # Termos Z_i Z_j
    for i in range(n_qubits):
        for j in range(i+1, n_qubits):
            if abs(J[i,j]) > 1e-10:
                c1, t1 = i // n_cidades, i % n_cidades
                c2, t2 = j // n_cidades, j % n_cidades
                sinal = "+" if J[i,j] > 0 else "-"
                termos.append(f"{sinal} {abs(J[i,j]):.2f} Z_{{{c1},{t1}}} Z_{{{c2},{t2}}}")
    
    # Mostra primeiros termos
    for termo in termos[:max_termos]:
        print(f"   {termo}")
    
    if len(termos) > max_termos:
        print(f"   ... (+{len(termos) - max_termos} termos adicionais)")
    
    print(f"\nTotal de termos: {len(termos)}")


mostrar_hamiltoniano_resumido(h_demo, J_demo, 3)

---
## 🔄 Célula 7: Algoritmo QAOA

### Estrutura do QAOA

O QAOA prepara o estado variacional através da aplicação alternada de dois operadores sobre o estado inicial de superposição uniforme:

$$|s\rangle = |+\rangle^{\otimes n^2} = H^{\otimes n^2} |0\rangle^{\otimes n^2}$$

### Operadores do Circuito

**1. Operador de Custo (Phase Operator):**
$$U_C(\gamma) = e^{-i \gamma \hat{H}_C}$$

**2. Operador de Mistura (Mixer):**
$$U_M(\beta) = e^{-i \beta \hat{H}_M}, \quad \text{com} \quad \hat{H}_M = \sum_{k=1}^{n^2} \hat{X}_k$$

### Estado Final

Após $p$ camadas (repetições):
$$|\psi(\vec{\gamma}, \vec{\beta})\rangle = \prod_{l=1}^{p} U_M(\beta_l) U_C(\gamma_l) |s\rangle$$

### Ciclo Híbrido

O otimizador clássico ajusta $(\vec{\gamma}, \vec{\beta})$ para minimizar:
$$\langle \psi(\vec{\gamma}, \vec{\beta}) | \hat{H}_C | \psi(\vec{\gamma}, \vec{\beta}) \rangle$$

In [ ]:
def criar_quadratic_program_tsp(D, A=None):
    """
    Cria o QuadraticProgram do Qiskit para o TSP.
    
    Implementa explicitamente:
    - Função objetivo: H_dist
    - Penalidades: H_p1 + H_p2
    """
    n = len(D)
    
    if A is None:
        A = np.max(D) * n + 1
    
    qp = QuadraticProgram(name=f'TSP_{n}_cidades')
    
    # Cria variáveis binárias x_{i,t}
    for i in range(n):
        for t in range(n):
            qp.binary_var(name=f'x_{i}_{t}')
    
    # ===== FUNÇÃO OBJETIVO =====
    # Termos quadráticos para H_dist + H_p1 + H_p2
    
    linear = {}
    quadratic = {}
    
    # H_dist: Σ d_ij * x_{i,t} * x_{j,t+1}
    for i in range(n):
        for j in range(n):
            if i != j:
                for t in range(n):
                    t_next = (t + 1) % n
                    var1 = f'x_{i}_{t}'
                    var2 = f'x_{j}_{t_next}'
                    key = (var1, var2) if var1 <= var2 else (var2, var1)
                    quadratic[key] = quadratic.get(key, 0) + D[i, j]
    
    # H_p1: A * Σ_i (Σ_t x_{i,t} - 1)²
    for i in range(n):
        for t in range(n):
            var = f'x_{i}_{t}'
            # Termo linear: -2A (de expandir quadrado)
            linear[var] = linear.get(var, 0) - 2 * A
            # Termo quadrático x²: +A (mas x² = x, então vai pro linear)
            linear[var] = linear.get(var, 0) + A
            
            # Termos cruzados: 2A * x_{i,t} * x_{i,t'}
            for t2 in range(t + 1, n):
                var2 = f'x_{i}_{t2}'
                key = (var, var2)
                quadratic[key] = quadratic.get(key, 0) + 2 * A
    
    # H_p2: A * Σ_t (Σ_i x_{i,t} - 1)²
    for t in range(n):
        for i in range(n):
            var = f'x_{i}_{t}'
            linear[var] = linear.get(var, 0) - 2 * A
            linear[var] = linear.get(var, 0) + A
            
            for i2 in range(i + 1, n):
                var2 = f'x_{i2}_{t}'
                key = (var, var2) if var <= var2 else (var2, var)
                quadratic[key] = quadratic.get(key, 0) + 2 * A
    
    # Define objetivo
    qp.minimize(linear=linear, quadratic=quadratic)
    
    return qp, A


if QISKIT_AVAILABLE:
    # Demonstração
    qp_demo, A_usado = criar_quadratic_program_tsp(graphs[3])
    print("\n📋 QuadraticProgram criado:")
    print(f"   Nome: {qp_demo.name}")
    print(f"   Variáveis: {qp_demo.get_num_vars()}")
    print(f"   Fator de penalidade A: {A_usado}")

---
## 🚀 Célula 8: Implementação Completa TSP-QAOA

In [ ]:
def calcular_custo_rota(rota, D):
    """
    Calcula o custo total de uma rota.
    """
    custo = 0
    for i in range(len(rota) - 1):
        custo += D[rota[i], rota[i + 1]]
    return custo


def brute_force_tsp(D):
    """
    Resolve TSP por força bruta.
    """
    n = len(D)
    melhor_rota = None
    melhor_custo = float('inf')
    
    for perm in permutations(range(1, n)):
        rota = (0,) + perm + (0,)
        custo = calcular_custo_rota(rota, D)
        if custo < melhor_custo:
            melhor_custo = custo
            melhor_rota = rota
    
    return melhor_rota, melhor_custo


def decodificar_solucao_tsp(x, n):
    """
    Decodifica o vetor binário x em uma rota.
    
    x[i*n + t] = 1 significa cidade i no tempo t
    """
    rota = []
    for t in range(n):
        for i in range(n):
            idx = i * n + t
            if idx < len(x) and x[idx] > 0.5:
                rota.append(i)
                break
    
    if len(rota) == n:
        rota.append(rota[0])  # Fecha o ciclo
        return tuple(rota)
    return None


def resolver_tsp_qaoa(D, reps=2, maxiter=150, verbose=True):
    """
    Resolve o TSP usando QAOA.
    
    Parâmetros:
    -----------
    D : np.array
        Matriz de distâncias
    reps : int
        Número de camadas p do QAOA
    maxiter : int
        Iterações máximas do otimizador clássico
    
    Retorna:
    --------
    rota : tuple
        Melhor rota encontrada
    custo : float
        Custo da rota
    info : dict
        Informações adicionais
    """
    if not QISKIT_AVAILABLE:
        return None, None, {'erro': 'Qiskit não disponível'}
    
    n = len(D)
    
    if verbose:
        print(f"\n⚛️ Executando QAOA para {n} cidades")
        print(f"   Qubits necessários: {n**2}")
        print(f"   Camadas (reps): {reps}")
        print(f"   Parâmetros variacionais: {2 * reps} (γ₁...γₚ, β₁...βₚ)")
    
    try:
        # Cria o problema
        qp, A = criar_quadratic_program_tsp(D)
        
        if verbose:
            print(f"   Fator de penalidade A: {A}")
        
        # Configura QAOA
        sampler = Sampler()
        optimizer = COBYLA(maxiter=maxiter)
        
        qaoa = QAOA(
            sampler=sampler,
            optimizer=optimizer,
            reps=reps
        )
        
        # Resolve
        algorithm = MinimumEigenOptimizer(qaoa)
        result = algorithm.solve(qp)
        
        # Decodifica
        x = result.x
        rota = decodificar_solucao_tsp(x, n)
        
        if rota is None:
            return None, None, {'erro': 'Solução inválida (restrições violadas)'}
        
        custo = calcular_custo_rota(rota, D)
        
        info = {
            'vetor_x': x,
            'fval_qubo': result.fval,
            'penalidade_A': A,
            'reps': reps
        }
        
        return rota, custo, info
        
    except Exception as e:
        return None, None, {'erro': str(e)}


print("✅ Funções de resolução carregadas!")

---
## 🎯 Célula 9: Execução Completa - Todas as Matrizes

In [ ]:
# Armazena resultados
resultados = {}

print("=" * 80)
print("TSP: COMPARAÇÃO BRUTE FORCE vs QAOA")
print("Implementação alinhada com a formulação matemática")
print("=" * 80)

for n_cidades, D in graphs.items():
    print(f"\n{'='*80}")
    print(f"📍 PROCESSANDO: {n_cidades} CIDADES")
    print(f"{'='*80}")
    
    print(f"\nMatriz de distâncias D:")
    print(D.astype(int))
    
    resultados[n_cidades] = {}
    
    # ===== BRUTE FORCE =====
    print(f"\n[1/2] 🔍 Brute Force Clássico...")
    inicio = time.time()
    rota_bf, custo_bf = brute_force_tsp(D)
    tempo_bf = time.time() - inicio
    
    print(f"      ✅ Rota ótima: {rota_bf}")
    print(f"      ✅ Custo: {custo_bf}")
    print(f"      ✅ Tempo: {tempo_bf:.6f}s")
    
    resultados[n_cidades]['brute_force'] = {
        'rota': rota_bf,
        'custo': custo_bf,
        'tempo': tempo_bf
    }
    
    # ===== QAOA =====
    print(f"\n[2/2] ⚛️ QAOA (Quantum Approximate Optimization)...")
    
    if QISKIT_AVAILABLE:
        inicio = time.time()
        rota_qaoa, custo_qaoa, info = resolver_tsp_qaoa(D, reps=2, maxiter=200)
        tempo_qaoa = time.time() - inicio
        
        if 'erro' in info:
            print(f"      ❌ Erro: {info['erro']}")
            resultados[n_cidades]['qaoa'] = {'erro': info['erro']}
        else:
            print(f"      ✅ Rota: {rota_qaoa}")
            print(f"      ✅ Custo: {custo_qaoa}")
            print(f"      ✅ Tempo: {tempo_qaoa:.4f}s")
            
            # Análise
            if custo_qaoa == custo_bf:
                print(f"      🎯 QAOA encontrou a solução ÓTIMA!")
            else:
                gap = ((custo_qaoa - custo_bf) / custo_bf) * 100
                print(f"      📈 Gap de otimalidade: {gap:.2f}%")
            
            resultados[n_cidades]['qaoa'] = {
                'rota': rota_qaoa,
                'custo': custo_qaoa,
                'tempo': tempo_qaoa,
                'info': info
            }
    else:
        print("      ❌ Qiskit não disponível")
        resultados[n_cidades]['qaoa'] = {'erro': 'Qiskit não disponível'}

print(f"\n{'='*80}")
print("✅ PROCESSAMENTO CONCLUÍDO!")
print("="*80)

---
## 📊 Célula 10: Tabela Resumo Final

In [ ]:
print("\n" + "=" * 100)
print("📊 TABELA RESUMO - BRUTE FORCE vs QAOA")
print("=" * 100)

print(f"\n{'n':<5} {'Qubits':<8} {'Rota BF':<18} {'Custo BF':<10} "
      f"{'Rota QAOA':<18} {'Custo QAOA':<12} {'Gap':<10} {'Status':<10}")
print("-" * 100)

for n in graphs.keys():
    bf = resultados[n]['brute_force']
    qaoa = resultados[n].get('qaoa', {})
    
    n_qubits = n ** 2
    rota_bf_str = str(bf['rota'])
    custo_bf = bf['custo']
    
    if 'erro' in qaoa:
        rota_qaoa_str = "N/A"
        custo_qaoa_str = "N/A"
        gap_str = "N/A"
        status = "❌"
    else:
        rota_qaoa_str = str(qaoa['rota'])
        custo_qaoa = qaoa['custo']
        custo_qaoa_str = str(int(custo_qaoa))
        
        if custo_qaoa == custo_bf:
            gap_str = "0%"
            status = "✅ Ótimo"
        else:
            gap = ((custo_qaoa - custo_bf) / custo_bf) * 100
            gap_str = f"+{gap:.1f}%"
            status = "📈"
    
    print(f"{n:<5} {n_qubits:<8} {rota_bf_str:<18} {int(custo_bf):<10} "
          f"{rota_qaoa_str:<18} {custo_qaoa_str:<12} {gap_str:<10} {status:<10}")

print("-" * 100)
print("\n📝 Legenda:")
print("   n = número de cidades")
print("   Qubits = n² (variáveis x_{i,t})")
print("   Gap = (Custo_QAOA - Custo_BF) / Custo_BF × 100%")

---
## 📈 Célula 11: Visualização das Rotas

In [ ]:
def plot_rota(ax, D, rota, titulo, cor='blue'):
    """
    Visualiza uma rota TSP.
    """
    n = len(D)
    angulos = np.linspace(0, 2 * np.pi, n, endpoint=False)
    pos = {i: (np.cos(a), np.sin(a)) for i, a in enumerate(angulos)}
    
    # Arestas background
    for i in range(n):
        for j in range(i + 1, n):
            ax.plot([pos[i][0], pos[j][0]], [pos[i][1], pos[j][1]], 
                    'lightgray', linewidth=0.5)
    
    # Rota
    if rota:
        for k in range(len(rota) - 1):
            i, j = rota[k], rota[k + 1]
            ax.plot([pos[i][0], pos[j][0]], [pos[i][1], pos[j][1]], 
                    color=cor, linewidth=2.5)
    
    # Nós
    for i, (x, y) in pos.items():
        c = 'red' if i == 0 else 'green'
        ax.scatter(x, y, s=400, c=c, edgecolors='black', zorder=5)
        ax.text(x, y, str(i), fontsize=10, ha='center', va='center', 
                fontweight='bold', color='white', zorder=6)
    
    ax.set_title(titulo, fontsize=10, fontweight='bold')
    ax.set_aspect('equal')
    ax.axis('off')


# Cria figura
fig, axes = plt.subplots(4, 2, figsize=(12, 18))

for idx, n in enumerate(graphs.keys()):
    D = graphs[n]
    bf = resultados[n]['brute_force']
    qaoa = resultados[n].get('qaoa', {})
    
    # Brute Force
    plot_rota(axes[idx, 0], D, bf['rota'],
              f"Brute Force (n={n})\nRota: {bf['rota']}\nCusto: {bf['custo']}",
              cor='blue')
    
    # QAOA
    if 'erro' not in qaoa:
        status = "✅" if qaoa['custo'] == bf['custo'] else f"Gap: {((qaoa['custo']-bf['custo'])/bf['custo'])*100:.1f}%"
        plot_rota(axes[idx, 1], D, qaoa['rota'],
                  f"QAOA (n={n}, {n**2} qubits)\nRota: {qaoa['rota']}\nCusto: {qaoa['custo']} {status}",
                  cor='purple')
    else:
        axes[idx, 1].text(0.5, 0.5, f"QAOA indisponível\n{qaoa.get('erro', '')}",
                          ha='center', va='center', fontsize=11,
                          transform=axes[idx, 1].transAxes)
        axes[idx, 1].set_title(f"QAOA (n={n})", fontsize=10, fontweight='bold')
        axes[idx, 1].axis('off')

plt.tight_layout()
plt.savefig('tsp_qaoa_comparacao.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n📁 Figura salva: tsp_qaoa_comparacao.png")

---
## 📐 Célula 12: Análise Detalhada do Mapeamento (Exemplo n=3)

In [ ]:
# Demonstração completa do pipeline para n=3
print("="*80)
print("📐 ANÁLISE DETALHADA DO PIPELINE TSP → QUBO → ISING → QAOA")
print("   Exemplo: n=3 cidades")
print("="*80)

D3 = graphs[3]
n = 3

# 1. Matriz de distâncias
print("\n1️⃣ MATRIZ DE DISTÂNCIAS D:")
print(D3.astype(int))

# 2. Variáveis
print("\n2️⃣ VARIÁVEIS DE DECISÃO x_{i,t}:")
print("   Cada x_{i,t} ∈ {0,1} indica se cidade i é visitada no tempo t")
mostrar_mapeamento_variaveis(3)

# 3. QUBO
print("\n3️⃣ CONSTRUÇÃO DA MATRIZ QUBO:")
Q, A, const = construir_qubo_manual(D3)

# 4. Ising
print("\n4️⃣ CONVERSÃO PARA HAMILTONIANO ISING:")
print(f"   Transformação: x_{{i,t}} → (I - Z_{{i,t}}) / 2")
h, J, offset = qubo_para_ising(Q, const)
mostrar_hamiltoniano_resumido(h, J, 3, max_termos=8)

# 5. Solução ótima
print("\n5️⃣ SOLUÇÃO ÓTIMA (Brute Force):")
rota_opt, custo_opt = brute_force_tsp(D3)
print(f"   Rota: {rota_opt}")
print(f"   Custo: {custo_opt}")

# Vetor x correspondente
print("\n   Vetor binário x correspondente:")
x_opt = np.zeros(n*n)
for t, cidade in enumerate(rota_opt[:-1]):
    idx = cidade * n + t
    x_opt[idx] = 1

print("   ", end="")
for i in range(n):
    for t in range(n):
        idx = i * n + t
        print(f"x_{{{i},{t}}}={int(x_opt[idx])} ", end="")
    print()
    print("   ", end="")

---
## 📝 Resumo da Correspondência Teoria ↔ Implementação

| Conceito Teórico | Implementação no Código |
|-----------------|-------------------------|
| Variável $x_{i,t}$ | `qp.binary_var(f'x_{i}_{t}')` |
| Função objetivo $C(\mathbf{x})$ | Termos quadráticos em `quadratic` |
| Penalidade $H_{p1}$ (cidade) | Loop sobre cidades com fator $A$ |
| Penalidade $H_{p2}$ (tempo) | Loop sobre tempos com fator $A$ |
| Fator $A$ | `A = max(D) * n + 1` |
| Mapeamento $x \to \frac{I-Z}{2}$ | `QuadraticProgramToQubo` (interno) |
| Hamiltoniano $\hat{H}_C$ | Operador Pauli via Qiskit |
| Operador $U_C(\gamma)$ | `QAOA` com `reps` camadas |
| Operador $U_M(\beta)$ | Mixer padrão (Pauli-X) |
| Otimizador clássico | `COBYLA(maxiter=...)` |
| Estado inicial $|+\rangle^{\otimes n^2}$ | Padrão do QAOA |